In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import random
import torchvision
import numpy as np, random
from PIL import Image
from matplotlib import pyplot as plt
import cv2
import glob
import os
from scipy import ndimage
from scipy.spatial import ConvexHull
plt.rcParams['figure.figsize'] = [10, 10]

Aponte os diretórios com as imagens a serem processadas e o diretório de resultados.

In [2]:
# data_dir = '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/'
data_dir = './sample_data'
out_dir = './sample_results'

Abaixo temos a definição da Unet modificada para também retornar um feature map intermediário que é utilizado como entrada pelo modelo de segmentação de texto. Eu escolhi essa modificação por que no momento que eu optei por criar também a segmentação de texto eu ja tinha os pesos para a Unet original bem treinados ,assim sendo eu não queria começar todo o treinamento do Zero.

In [3]:
class UNetVgg(torch.nn.Module):
    def __init__(self, nClasses):
        super(UNetVgg, self).__init__()

        vgg16pre = torchvision.models.vgg16(pretrained=True)
        self.vgg0 = torch.nn.Sequential(*list(vgg16pre.features.children())[:4])
        self.vgg1 = torch.nn.Sequential(*list(vgg16pre.features.children())[4:9])
        self.vgg2 = torch.nn.Sequential(*list(vgg16pre.features.children())[9:16])
        self.vgg3 = torch.nn.Sequential(*list(vgg16pre.features.children())[16:23])
        self.vgg4 = torch.nn.Sequential(*list(vgg16pre.features.children())[23:30])



        self.smooth0 = torch.nn.Sequential(
                torch.nn.Conv2d(128, 64, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True),
                torch.nn.Conv2d(64, 64, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True)
                )
        self.smooth1 = torch.nn.Sequential(
                torch.nn.Conv2d(256, 64, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True),
                torch.nn.Conv2d(64, 64, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True)
                )
        self.smooth2 = torch.nn.Sequential(
                torch.nn.Conv2d(512, 128, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True),
                torch.nn.Conv2d(128, 128, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True)
                )
        self.smooth3 = torch.nn.Sequential(
                torch.nn.Conv2d(1024, 256, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True),
                torch.nn.Conv2d(256, 256, kernel_size=(3,3), stride=1, padding=(1, 1)),
                torch.nn.ReLU(True)
                )

        self.final = torch.nn.Conv2d(64, nClasses, kernel_size=1, stride=1, padding=0)

    def forward(self, x):

        feat0 = self.vgg0(x)
        feat1 = self.vgg1(feat0)
        feat2 = self.vgg2(feat1)
        feat3 = self.vgg3(feat2)
        feat4 = self.vgg4(feat3)

        _,_,H,W = feat3.size()
        up3 = torch.nn.functional.interpolate(feat4, size=(H,W), mode='bilinear', align_corners=False)
        concat3 = torch.cat([feat3, up3], 1)
        end3 = self.smooth3(concat3)

        _,_,H,W = feat2.size()
        up2 = torch.nn.functional.interpolate(end3, size=(H,W), mode='bilinear', align_corners=False)
        concat2 = torch.cat([feat2, up2], 1)
        end2 = self.smooth2(concat2)

        _,_,H,W = feat1.size()
        up1 = torch.nn.functional.interpolate(end2, size=(H,W), mode='bilinear', align_corners=False)
        concat1 = torch.cat([feat1, up1], 1)
        end1 = self.smooth1(concat1)

        _,_,H,W = feat0.size()
        up0 = torch.nn.functional.interpolate(end1, size=(H,W), mode='bilinear', align_corners=False)
        concat0 = torch.cat([feat0, up0], 1)
        end0 = self.smooth0(concat0)

        return self.final(end0), end2


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nClasses = 2
model = UNetVgg(nClasses).to(device)
model_name = 'cleanner.pth'


seg_model = torch.nn.Sequential(
        torch.nn.Conv2d(128, 128, kernel_size=(3,3), stride=1, padding=(1, 1)),
        torch.nn.ReLU(True),
        torch.nn.Conv2d(128, 128, kernel_size=(3,3), stride=1, padding=(1, 1)),
        torch.nn.ReLU(True),
        torch.nn.Conv2d(128, nClasses, kernel_size=1, stride=1, padding=0)
        ).to(device)
seg_name = 'seg.pth'


try:
    model.load_state_dict(torch.load(model_name))
except:
    raise "Unable to load model"
model = model.eval()

try:
    seg_model.load_state_dict(torch.load(seg_name))
except:
    raise "Unable to load model"
    
seg_model = seg_model.eval()

In [5]:
list_files = []
for ext in ['png', 'jpeg', 'jpg', 'tif']:
    list_files += glob.glob(data_dir +'/*.' + ext)
list_files

['/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/198.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/144.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/29.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/95.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/51.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/110.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/122.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/66.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/53.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/153.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCleanup/noisy_data/161.png',
 '/personal/nuveo_test/cv-test/candidate-data/01-DocumentCl

Abaixo a função que usa os modelos propostos para fazer a inferencia do texto limpo.  
As imagens de entrada são invertidas de forma que os caracters ocupem os pixels com maior valor numérico, este é o unico preprocessamento realizado nos dados.  
O parametro with_seg_attention diz se vamos utilizar a segmentação de area de texto como uma espécie de "visual attention" para a segmentação de caracters.

In [6]:
def apply_model_to_image(net, np_image, with_seg_attention=True):
    
    if len(np_image.shape) == 2:
        np_image = np.dstack((np_image, np_image, np_image))
#     np_image = cv2.normalize(np_image,  None, 0, 255, cv2.NORM_MINMAX)
    np_image = 255 - np_image
    
    np_image = np_image.transpose(2, 0, 1).astype('f4')
    np_image = np_image[np.newaxis,...]
    
    label_out, seg_feature = model(torch.from_numpy(np_image).float().to(device))

    seg_out = seg_model(seg_feature)


    _,_,H,W = label_out.size()
#     print(H,W)
    seg_out = torch.nn.functional.interpolate(seg_out, size=(H,W), mode='bilinear', align_corners=False)
    
    attention = torch.nn.functional.softmax(seg_out, dim = 1)
    channelA = label_out[:,1,:,:] * attention[:,1,:,:]

    channelB = label_out[:,0,:,:] + attention[:,0,:,:]
    label_out  = torch.stack((channelB, channelA), 1)
    
    label_out = torch.nn.functional.softmax(label_out, dim = 1)
    label_out = label_out.cpu().detach().numpy()
    label_out = np.squeeze(label_out)

    seg_out = attention.cpu().detach().numpy()
    seg_out = np.squeeze(seg_out)  
    
    label_out = np.argmax(label_out, axis=0).astype(np.uint8) 
    seg_out = np.argmax(seg_out, axis=0).astype(np.uint8) 
    
    if with_seg_attention:
        label_out = label_out * seg_out

    return (1-label_out) * 255


Essas funções foram retiradas do site PyImageSearch

In [7]:
# reference PyImageSearch

def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):

    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    meanr = np.mean(rect, axis=0)
    crect = rect - meanr
    crect = crect * 1.05
    
    rect= crect + meanr
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [8]:
def get_angle_from_principal_component(np_image):
    points = np.argwhere(np_image == 0)

    mean_points = points.mean(axis = 0)

    center_data = points - mean_points

    covm = center_data.T.dot(center_data)  * 1.0/float(center_data.shape[0] - 1)

    ev , eig = np.linalg.eig(covm)
    top_eig = eig[:,np.argmax(ev)]
    ang = np.arctan(top_eig[0]/top_eig[1])
    return ang

Abaixo acontece o processamento de cada uma das images. Note que o modelo é aplicado uma primeira vez onde ambos os modelos são utilizados tanto o segmenta os caracters quanto o que segmenta blocos de texto, na segunda vez apenas o medelo principal que segmenta os caracters é utilizado.  
Aqui foi utilizado a função do OpenCV cv2.minAreaRect como fonte dos pontos utilizados para fazer o Warp da imagem essa função retorna um retangulo com um angulo de rotação. Acontece que em muitos casos o texto ocupa uma região que é semelhante a um paralelograma o que faz com que o resultado não fique perfeito.  
Aqui para se faz necessário a criação de uma função de fit de um quadritero arbitrário o que deixaria o resultado ainda melhor. Eu optei por não implementar essa função apenas por conta do tempo que eu tinha disponivel.

In [9]:
def process_image(net, np_image):
    rst = apply_model_to_image(net, np_image)
    
#     ang = get_angle_from_principal_component(rst)
#     rot_image = ndimage.rotate(np_image, np.rad2deg(ang))
    
#     rst = apply_model_to_image(net, rot_image)
    
    kernel = np.ones((7,2),np.uint8)
    closing = cv2.morphologyEx(rst, cv2.MORPH_CLOSE, kernel)
    closing = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, kernel)
#     plt.imshow(closing)
#     plt.show()
    img_points = (~(closing == 255)*255).astype('u1')
    img_points[:10,...] = 0 
    img_points[-10:,...] = 0 
    img_points[..., :10] = 0 
    img_points[..., -10:] = 0
    
    points = np.argwhere(img_points == 255)

    hull = ConvexHull(points)
    cnt = points[hull.vertices]
    cnt = cnt[...,::-1]

    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    np_image = four_point_transform(np_image, box)
    
    
    rst = apply_model_to_image(net, np_image, False)
    return rst

Processa cada uma das imagens e salva os resultados

In [10]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
for img in list_files:
    np_image = np.asarray(Image.open(img))
    rst = process_image(model, np_image)
    file_name = os.path.basename(img)
    Image.fromarray(rst).save(os.path.join(out_dir , file_name) )